# Homework no: 14
## Name: Anggara Putra Pratama - 1101174240
### Reference = Muhammad Kemal Hernandi - 1101174169
#GRU

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

## 2. Define the Dataset

In [14]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import csv
import gzip


class NameDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self, is_train_set=False):
        filename = '/content/drive/MyDrive/Colab Notebooks/data/names_train.csv.gz' if is_train_set else '/content/drive/MyDrive/Colab Notebooks/data/names_test.csv.gz'
        with gzip.open(filename, "rt") as f:
            reader = csv.reader(f)
            rows = list(reader)

        self.names = [row[0] for row in rows]
        self.countries = [row[1] for row in rows]
        self.len = len(self.countries)

        self.country_list = list(sorted(set(self.countries)))

    def __getitem__(self, index):
        return self.names[index], self.countries[index]

    def __len__(self):
        return self.len

    def get_countries(self):
        return self.country_list

    def get_country(self, id):
        return self.country_list[id]

    def get_country_id(self, country):
        return self.country_list.index(country)

# Test the loader
if __name__ == "__main__":
    dataset = NameDataset(False)
    print(dataset.get_countries())
    print(dataset.get_country(3))
    print(dataset.get_country_id('Korean'))

    train_loader = DataLoader(dataset=dataset,
                              batch_size=10,
                              shuffle=True)

    print(len(train_loader.dataset))
    for epoch in range(2):
        for i, (names, countries) in enumerate(train_loader):
            # Run your training process
            print(epoch, i, "names", names, "countries", countries)


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
Dutch
11
6700
0 0 names ('Toman', 'Pelevin', 'Sfakianos', 'Roy', 'Borovsky', 'Morava', 'Adokhin', 'Govyrin', 'Matzkov', 'Truhachev') countries ('Russian', 'Russian', 'Greek', 'French', 'Czech', 'Czech', 'Russian', 'Russian', 'Russian', 'Russian')
0 1 names ('Shammas', 'Babakhanov', 'Tumanyan', 'Galanopoulos', 'Kalb', 'Pavlushin', 'Weichert', 'Dolejsi', 'Tuma', 'Zenischev') countries ('Arabic', 'Russian', 'Russian', 'Greek', 'Arabic', 'Russian', 'Czech', 'Czech', 'Arabic', 'Russian')
0 2 names ('Roper', 'Wasem', 'Emmons', 'Krawiec', 'Tukvachinsky', 'Cotterill', 'Ryzhko', 'Gunston', 'Avetisov', 'Nazario') countries ('English', 'Arabic', 'English', 'Czech', 'Russian', 'English', 'Russian', 'English', 'Russian', 'Italian')
0 3 names ('Sweeting', 'Turuhin', 'Averkoff', 'Stoppelbein', 'Yakutin', 'Bakh

## 3. Define the Parameters and DataLoaders

In [15]:
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 100

test_dataset = NameDataset(is_train_set=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=True)


train_dataset = NameDataset(is_train_set=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

N_COUNTRIES = len(train_dataset.get_countries())
print(N_COUNTRIES, "countries")
N_CHARS = 128  # ASCII

18 countries


## 4. Define Some Utility Functions

In [16]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def create_variable(tensor):
    # Do cuda() before wrapping with variable
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

## 5. Define the Pad Sequences and Sort the Tensor

In [17]:
def pad_sequences(vectorized_seqs, seq_lengths, countries):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)

    # Sort tensors by their length
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]

    # Also sort the target (countries) in the same order
    target = countries2tensor(countries)
    if len(countries):
        target = target[perm_idx]

    # Return variables
    # DataParallel requires everything to be a Variable
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)

## 6. Create Necessary Variables, Lengths, and Target

In [18]:
def make_variables(names, countries):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, countries)


def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)


def countries2tensor(countries):
    country_ids = [train_dataset.get_country_id(
        country) for country in countries]
    return torch.LongTensor(country_ids)

## 7. Define The Network Structures (RNN)

In [19]:
class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, seq_lengths):
        # Note: we run this all at once (over the whole input sequence)
        # input shape: B x S (input size)
        # transpose to make S(sequence) x B (batch)
        input = input.t()
        batch_size = input.size(1)

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        # Embedding S x B -> S x B x I (embedding size)
        embedded = self.embedding(input)

        # Pack them up nicely
        gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())

        # To compact weights again call flatten_parameters().
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)

        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden[-1])
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)


## 8. Create The Training and Testing Cycle Hidden Size

In [20]:
# Train cycle
def train():
    total_loss = 0

    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)

        loss = criterion(output, target)
        total_loss += loss.item()

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss

# Test cycle
def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))


if __name__ == '__main__':

    classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)

    if torch.cuda.is_available():
        classifier.cuda()

    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        train()

        # Testing
        test()

        # Testing several samples
        test("Sung")
        test("Jungwoo")
        test("Soojin")
        test("Nako")

Training for 100 epochs...
 0s] Train Epoch: 1 [2560/13374 (19%)]	Loss: 598.46
 0s] Train Epoch: 1 [5120/13374 (38%)]	Loss: 530.99
 0s] Train Epoch: 1 [7680/13374 (57%)]	Loss: 496.66
 0s] Train Epoch: 1 [10240/13374 (77%)]	Loss: 470.86
 0s] Train Epoch: 1 [12800/13374 (96%)]	Loss: 449.24
evaluating trained model ...

Test set: Accuracy: 4208/6700 (63%)

Sung is English
Jungwoo is Russian
Soojin is Arabic
Nako is Arabic
 1s] Train Epoch: 2 [2560/13374 (19%)]	Loss: 330.95
 1s] Train Epoch: 2 [5120/13374 (38%)]	Loss: 321.37
 1s] Train Epoch: 2 [7680/13374 (57%)]	Loss: 314.48
 1s] Train Epoch: 2 [10240/13374 (77%)]	Loss: 309.56
 1s] Train Epoch: 2 [12800/13374 (96%)]	Loss: 302.15
evaluating trained model ...

Test set: Accuracy: 4720/6700 (70%)

Sung is Arabic
Jungwoo is Russian
Soojin is Russian
Nako is Japanese
 1s] Train Epoch: 3 [2560/13374 (19%)]	Loss: 253.97
 2s] Train Epoch: 3 [5120/13374 (38%)]	Loss: 255.62
 2s] Train Epoch: 3 [7680/13374 (57%)]	Loss: 254.81
 2s] Train Epoch: 3 [10

## 8. Create The Training and Testing Cycle

In [21]:
# Train cycle
def train():
    total_loss = 0

    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)

        loss = criterion(output, target)
        total_loss += loss.item()

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss

# Test cycle
def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))


if __name__ == '__main__':

    classifier = RNNClassifier(N_CHARS, 250, N_COUNTRIES, N_LAYERS)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)

    if torch.cuda.is_available():
        classifier.cuda()

    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    print("Training for %d epochs..." % 100)
    for epoch in range(1, 100 + 1):
        # Train cycle
        train()

        # Testing
        test()

        # Testing several samples
        test("Sung")
        test("Jungwoo")
        test("Soojin")
        test("Nako")


Training for 100 epochs...
 0s] Train Epoch: 1 [2560/13374 (19%)]	Loss: 518.32
 0s] Train Epoch: 1 [5120/13374 (38%)]	Loss: 449.89
 0s] Train Epoch: 1 [7680/13374 (57%)]	Loss: 408.77
 0s] Train Epoch: 1 [10240/13374 (77%)]	Loss: 382.45
 0s] Train Epoch: 1 [12800/13374 (96%)]	Loss: 357.72
evaluating trained model ...

Test set: Accuracy: 4755/6700 (71%)

Sung is Chinese
Jungwoo is Russian
Soojin is Japanese
Nako is Japanese
 1s] Train Epoch: 2 [2560/13374 (19%)]	Loss: 238.82
 1s] Train Epoch: 2 [5120/13374 (38%)]	Loss: 237.17
 1s] Train Epoch: 2 [7680/13374 (57%)]	Loss: 229.44
 1s] Train Epoch: 2 [10240/13374 (77%)]	Loss: 222.20
 1s] Train Epoch: 2 [12800/13374 (96%)]	Loss: 215.24
evaluating trained model ...

Test set: Accuracy: 5246/6700 (78%)

Sung is Chinese
Jungwoo is Russian
Soojin is Russian
Nako is Japanese
 2s] Train Epoch: 3 [2560/13374 (19%)]	Loss: 166.93
 2s] Train Epoch: 3 [5120/13374 (38%)]	Loss: 166.40
 2s] Train Epoch: 3 [7680/13374 (57%)]	Loss: 163.03
 2s] Train Epoch: 